# Full Chromatin Simulation with Adaptable Refinement

In this notebook, we demonstrate a Monte Carlo simulation of chromatin organization where the level of detail is adaptibly confined to improve convergence onto a globally optimal configuration.

### Import Modules

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
import os
import sys
from inspect import getmembers, isfunction

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
cwd = os.getcwd()
parent_dir = cwd + "/../.."
sys.path.insert(1, parent_dir)
os.chdir(parent_dir)

print("Root Directory: ")
print(os.getcwd())

In [ ]:
import chromo.mc as mc
from chromo.polymers import Chromatin
import chromo.binders
from chromo.fields import UniformDensityField
import chromo.mc.mc_controller as ctrl
from chromo.util.reproducibility import get_unique_subfolder_name
from chromo.util.poly_paths import gaussian_walk
import chromo.util.rediscretize as rd
import chromo.util.mu_schedules as ms

### Generate Initial Chromatin Fiber

In [ ]:
# Binders
hp1 = chromo.binders.get_by_name("HP1")
prc1 = chromo.binders.get_by_name("PRC1")

hp1.chemical_potential = -0.4
prc1.chemical_potential = -0.4

binders = chromo.binders.make_binder_collection([hp1, prc1])

In [ ]:
# Confinement
confine_type = "Spherical"
confine_length = 900

In [ ]:
# Polymer
num_beads = 393216
bead_spacing = 16.5
chem_mods_path = np.array([
    "chromo/chemical_mods/HNCFF683HCZ_H3K9me3_methyl.txt",
    "chromo/chemical_mods/ENCFF919DOR_H3K27me3_methyl.txt"
])
chemical_mods = Chromatin.load_seqs(chem_mods_path)[:num_beads]
states = np.zeros(chemical_mods.shape, dtype=int)
p = Chromatin.confined_gaussian_walk(
    'Chr-1',
    num_beads,
    bead_length=bead_spacing,
    states=states,
    confine_type=confine_type,
    confine_length=confine_length,
    binder_names=np.array(['HP1', 'PRC1']),
    chemical_mods=chemical_mods,
    chemical_mod_names=np.array(['H3K9me3', 'H3K27me3'])
)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(p.r[:,0], p.r[:,1], p.r[:,2], s=0.5, alpha=0.5)
ax.set_xticks(np.arange(-900, 901, 300))
ax.set_yticks(np.arange(-900, 901, 300))
ax.set_zticks(np.arange(-900, 901, 300))
plt.show()

In [ ]:
# Field
n_accessible = 63
n_buffer = 2
n_bins_x = n_accessible + n_buffer
x_width = 2 * confine_length * (1 + n_buffer/n_accessible)
n_bins_y = n_bins_x
y_width = x_width
n_bins_z = n_bins_x
z_width = x_width
udf = UniformDensityField(
    [p], binders, x_width, n_bins_x, y_width,
    n_bins_y, z_width, n_bins_z, confine_type=confine_type,
    confine_length=confine_length, chi=1,
    assume_fully_accessible=1, fast_field=1, n_points=1000
)

### Coarse-Grain the Original Polymer

In [ ]:
cg_factor = 15

In [ ]:
p_cg = rd.get_cg_chromatin(
    polymer = p,
    cg_factor = cg_factor,
    name_cg = "Chr_CG"
)

In [ ]:
print("Average Coarse-Grained Bead Separation: ")
print(round(np.average(np.linalg.norm(np.diff(p_cg.r, axis=0), axis=1)), 3))

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(p_cg.r[:,0], p_cg.r[:,1], p_cg.r[:,2], s=10, alpha=0.5)
ax.set_xticks(np.arange(-900*cg_factor**(1/3), 900*cg_factor**(1/3)+1, 100))
ax.set_yticks(np.arange(-900*cg_factor**(1/3), 900*cg_factor**(1/3)+1, 100))
ax.set_zticks(np.arange(-900*cg_factor**(1/3), 900*cg_factor**(1/3)+1, 100))
plt.show()

In [ ]:
udf_cg = rd.get_cg_udf(
    udf_refined_dict = udf.dict_,
    binders_refined = binders,
    cg_factor = cg_factor,
    polymers_cg = [p_cg]
)

In [ ]:
binders_cg = rd.get_cg_binders(
    binders_refined = binders,
    cg_factor = cg_factor
)

### Equilibrate the Coarse-Grained Polymer

In [ ]:
amp_bead_bounds, amp_move_bounds = mc.get_amplitude_bounds([p_cg])
num_snapshots = 200
mc_steps_per_snapshot = 3000

# Create a list of mu schedules, which are defined in another file
schedules = [func[0] for func in getmembers(ms, isfunction)]
select_schedule = "linear_step_for_negative_cp"
mu_schedules = [
    ms.Schedule(getattr(ms, func_name)) for func_name in schedules
]
mu_schedules = [sch for sch in mu_schedules if sch.name == select_schedule]

In [ ]:
polymers_cg = mc.polymer_in_field(
    [p_cg],
    binders_cg,
    udf_cg,
    mc_steps_per_snapshot,
    num_snapshots,
    amp_bead_bounds,
    amp_move_bounds,
    output_dir='output',
    mu_schedule=mu_schedules[0],
    random_seed=np.random.randint(0, 1E5)
)

In [ ]:
p_cg = polymers_cg[0]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(p_cg.r[:,0], p_cg.r[:,1], p_cg.r[:,2], s=10, alpha=0.5)
plt.show()

In [ ]:
print("Average Coarse-Grained Bead Separation: ")
print(round(np.average(np.linalg.norm(np.diff(p_cg.r, axis=0), axis=1)), 3))

### Refine the Coarse-Grained Polymer

In [ ]:
n_bind_eq = 1000000
p_refine, udf_refine = rd.refine_chromatin(
    polymer_cg = p_cg,
    num_beads_refined = num_beads,
    bead_spacing = bead_spacing,
    chemical_mods = chemical_mods,
    udf_cg = udf_cg,
    binding_equilibration = n_bind_eq,
    name_refine = "Chr_refine",
    output_dir = "output"
)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(p_refine.r[:,0], p_refine.r[:,1], p_refine.r[:,2], s=0.5, alpha=0.5)
ax.set_xticks(np.arange(-900, 901, 300))
ax.set_yticks(np.arange(-900, 901, 300))
ax.set_zticks(np.arange(-900, 901, 300))
plt.show()

### Equilibrate the Refined Polymer

In [ ]:
amp_bead_bounds, amp_move_bounds = mc.get_amplitude_bounds([p_refine])
num_snapshots = 200
mc_steps_per_snapshot = 10000

# Create a list of mu schedules, which are defined in another file
schedules = [func[0] for func in getmembers(ms, isfunction)]
select_schedule = "linear_step_for_negative_cp_mild"
mu_schedules = [
    ms.Schedule(getattr(ms, func_name)) for func_name in schedules
]
mu_schedules = [sch for sch in mu_schedules if sch.name == select_schedule]

In [ ]:
polymers_refined = mc.polymer_in_field(
    [p_refine],
    binders,
    udf_refine,
    mc_steps_per_snapshot,
    num_snapshots,
    amp_bead_bounds,
    amp_move_bounds,
    output_dir='output',
    mu_schedule=mu_schedules[0],
    random_seed=np.random.randint(0, 1E5)
)

In [ ]:
p_refine = polymers_refined[0]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(p_refine.r[:,0], p_refine.r[:,1], p_refine.r[:,2], s=0.5, alpha=0.5)
ax.set_xticks(np.arange(-900, 901, 300))
ax.set_yticks(np.arange(-900, 901, 300))
ax.set_zticks(np.arange(-900, 901, 300))
plt.show()

In [ ]:
print("Average Coarse-Grained Bead Separation: ")
print(round(np.average(np.linalg.norm(np.diff(p_refine.r, axis=0), axis=1)), 3))